In [1]:
import numpy as np
#函数1
def generate_state(I, L, W, M, x, H, lambd):
    # Generate matrix (n_il)
    S=[]
    n_il = np.zeros((I, L), dtype=int)
    for i in range(I):
        for l in range(L):
            n_il[i, l] = np.random.poisson(lambd[i, l])
    S0=n_il.tolist()
    
    S1=[]
    for m in range(M):
        w_m = np.random.randint(0,W+1)
        if w_m == W:
            i_m = H[m]
        else:
            i_m = np.random.randint(1, I+1)
        S1.append((i_m, w_m))
    S=(S0,S1)    

    return S

In [2]:
I = 5
L = 5
W = 6
M = 7
x = 2
H = [1,2,3,2,4,5,7]
lambd = np.random.rand(I, L)
state = generate_state(I, L, W, M, x, H, lambd)
print(state)

([[1, 0, 0, 0, 2], [2, 2, 0, 0, 1], [0, 0, 0, 1, 2], [2, 1, 0, 0, 0], [2, 0, 0, 0, 2]], [(1, 6), (5, 0), (1, 1), (4, 1), (4, 6), (5, 1), (5, 3)])


In [3]:
def aggreg_state(S, Z, X, M, I, L,W):
    #函数2
    barM = sum([1 for i in range(M) if S[1][i][1] != W])  
    cluster = split_list(I, Z)
    num_cluster = divide_reminder(I, Z)
    g = num_cluster*[0]
    
    # 状态的第二部分压缩
    for z in range(num_cluster):
        e_z = sum([1 for m in range(M) if S[1][m][0] in cluster[z]])
        if e_z == 0:
            g[z] = 0
        elif e_z <= barM / num_cluster:
            g[z] = 1
        else:
            g[z] = 2
    w = S[1][0][1]    
    
    #状态的第一部分压缩
    N=[[0] * L for _ in range(num_cluster)]
    for z in range(num_cluster):
        for l in range(L):
            N[z][l] = min(X, sum([S[0][i-1][l] for i in cluster[z]]))
            
    barS =(N,g,w)        
    return barS

def split_list(I, Z):
    arr = list(range(1, I + 1))
    return split_array_given_array(arr, Z)

def split_array_given_array(arr, Z):
    result = []
    quotient = len(arr) // Z  # 整除的商
    remainder = len(arr) % Z  # 余数
    
    # 划分整除部分的数组
    for i in range(quotient):
        sub_array = arr[i * Z:(i + 1) * Z]
        result.append(sub_array)
    
    # 单独处理余数部分的数组
    if remainder > 0:
        sub_array = arr[-remainder:]
        result.append(sub_array)  
    return result

def divide_reminder(num, divisor):
    quotient = num // divisor  # 整除的商
    remainder = num % divisor  # 余数
    
    if remainder > 0:
        quotient += 1    
    return quotient

In [4]:
Z=3
X=3
M=7
I=5
L=5
W=6
barS=aggreg_state(state, Z, X, M, I, L, W)
print(barS)

([[3, 2, 0, 1, 3], [3, 1, 0, 0, 2]], [1, 2], 6)


In [5]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [16]:
from itertools import combinations_with_replacement, permutations
import random
#函数3a

def matrix_dic(s0):  #二维列表转为一维字典
    dic = {}
    for i, row in enumerate(s0):
        for j, value in enumerate(row):
            dic[(i+1, j+1)] = value
    return dic

def decision_space(s0, M):
    kk=matrix_dic(s0)
    b=list(kk.keys())
    a = []
    b1=list(reversed(b))
    for subset in combinations_with_replacement(b, M):
        a.append(list(subset))
    for subset in combinations_with_replacement(b1, M):
        a.append(list(subset))
    unique_sublists = set(tuple(sublist) for sublist in a)
    a = [list(sublist) for sublist in unique_sublists]
    return a

def change_val(b, c, d):
    a = b.copy()  
    for i in range(len(c)):
        a[c[i]] = d[i]  
    return a

def new_decision(hlist,llist,s,s0,M,W):
    zerow=[]
    for i in range(M):
        if s[1][i][1] == W:
            zerow.append(i)
    inhome = [hlist[k] for k in zerow]
    dec0 = [(x,0) for x in inhome]
    de=decision_space(s0, M)
    new_dec=[]
    for j in range(len(de)):
        n_dec = change_val(de[j],zerow,dec0)
        new_dec.append(n_dec)
    a = matrix_dic(s0)
    keys_list = list(a.keys()) 
    #de=new_decision(hlist,s,s0,M,W)
    a_dec=[]
    for j in range(len(new_dec)):
        for ke in keys_list:                    #重复值大于字典的value,则去掉
            if new_dec[j].count(ke) <= a[ke]:
                continue
            a_dec.append(new_dec[j]) 
    final_dec=[] 
    for k in range(len(new_dec)):
        for g in range(len(new_dec[k])):
            if new_dec[k][g][1] >= llist[g]:
                continue
        final_dec.append(new_dec[k])
    return final_dec


In [17]:
hlist = random.choices(range(1, 5), k=7)  #函数3代入参数
llist = [1,2,3,4,4,5,5]
s = state
s0=state[0]
M=7
W=6
DDD = new_decision(hlist,llist,s,s0,M,W)
print(DDD[:10])


[[(3, 0), (5, 3), (5, 1), (3, 2), (2, 0), (1, 3), (1, 1)], [(3, 0), (4, 2), (4, 1), (3, 2), (2, 0), (1, 3), (1, 2)], [(3, 0), (1, 4), (1, 4), (4, 4), (2, 0), (5, 1), (5, 5)], [(3, 0), (5, 3), (4, 4), (4, 2), (2, 0), (4, 1), (2, 5)], [(3, 0), (4, 3), (4, 2), (3, 2), (2, 0), (2, 3), (1, 2)], [(3, 0), (2, 3), (3, 3), (5, 1), (2, 0), (5, 3), (5, 5)], [(3, 0), (4, 5), (4, 5), (4, 1), (2, 0), (2, 1), (1, 4)], [(3, 0), (4, 1), (3, 1), (2, 2), (2, 0), (2, 1), (1, 3)], [(3, 0), (4, 3), (3, 4), (3, 1), (2, 0), (2, 2), (1, 5)], [(3, 0), (2, 2), (2, 1), (2, 1), (2, 0), (2, 1), (1, 2)]]
